In [1]:
import os
from io import BytesIO
import pandas as pd
import requests

In [2]:
from expiration_downloader import ExpirationDownloader

In [3]:
api_key = os.environ.get('QUANDL_API_KEY')

In [4]:
def fetch_table(api_key, show_progress=False):
    """
    Return quandl spec file with a list of all available contracts.
    """
    if show_progress:
        log.info('Downloading metadata file from quandl')

    r = requests.get('https://www.quandl.com/api/v3/databases/CME/metadata?api_key={}'.format(api_key))
    r.raise_for_status()
    return pd.read_csv(BytesIO(r.content), compression='zip', parse_dates=['from_date', 'to_date'])

In [5]:
table = fetch_table(api_key)

In [6]:
table.head()

,code,name,description,refreshed_at,from_date,to_date
0,0DF2015,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-12-28
1,0DF2016,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-02-02
2,0DF2017,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2017-02-03 01:05:31,2014-11-21,2017-02-02
3,0DF2018,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-02-03 01:07:36,2014-11-21,2018-02-02
4,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-10-31 00:11:23,2015-02-12,2018-10-30


In [7]:
d = ExpirationDownloader(table)

In [8]:
d.data.head()

,symbol,name,description,refreshed_at,from_date,to_date,year,root_symbol,exch_symbol,contract month,first trade,expiration_date,settlement
0,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-10-31 00:11:23,2015-02-12,2018-10-30,2019,0D,0DF19,January 2019 Mini European 3.5% Fuel Oil Barge...,2015-01-02,2019-01-31,2019-02-04
1,0DF2020,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-10-31 00:11:23,2017-12-04,2018-10-30,2020,0D,0DF20,January 2020 Mini European 3.5% Fuel Oil Barge...,2016-01-04,2020-01-31,2020-02-04
2,0DG2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-10-31 00:11:23,2015-02-12,2018-10-30,2019,0D,0DG19,February 2019 Mini European 3.5% Fuel Oil Barg...,2015-01-02,2019-02-28,2019-03-04
3,0DG2020,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-10-31 00:11:23,2017-12-04,2018-10-30,2020,0D,0DG20,February 2020 Mini European 3.5% Fuel Oil Barg...,2016-01-04,2020-02-28,2020-03-03
4,0DH2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-10-31 00:11:23,2015-02-12,2018-10-30,2019,0D,0DH19,March 2019 Mini European 3.5% Fuel Oil Barges ...,2015-01-02,2019-03-29,2019-04-02


In [9]:
d.data.count()

symbol             5542
name               5542
description        5542
refreshed_at       5542
from_date          5542
to_date            5542
year               5542
root_symbol        5542
exch_symbol        5542
contract month     5542
first trade        5542
expiration_date    5542
settlement         5542
dtype: int64

In [10]:
d.data.columns

Index(['symbol', 'name', 'description', 'refreshed_at', 'from_date', 'to_date',
       'year', 'root_symbol', 'exch_symbol', 'contract month', 'first trade',
       'expiration_date', 'settlement'],
      dtype='object')

In [11]:
table.columns

Index(['code', 'name', 'description', 'refreshed_at', 'from_date', 'to_date'], dtype='object')

In [12]:
table.head()

,code,name,description,refreshed_at,from_date,to_date
0,0DF2015,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-12-28
1,0DF2016,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-02-02
2,0DF2017,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2017-02-03 01:05:31,2014-11-21,2017-02-02
3,0DF2018,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-02-03 01:07:36,2014-11-21,2018-02-02
4,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-10-31 00:11:23,2015-02-12,2018-10-30


In [13]:
table.count()

code            164721
name            164721
description     164721
refreshed_at    164721
from_date       164721
to_date         164721
dtype: int64

In [14]:
a = pd.read_csv('C:/Users/Tomek/zipline/debug/meta_files/CME_metadata.csv')

In [15]:
a.head()

,code,name,description,refreshed_at,from_date,to_date
0,0DF2015,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-12-28
1,0DF2016,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-02-02
2,0DF2017,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2017-02-03 01:05:31,2014-11-21,2017-02-02
3,0DF2018,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-02-03 01:07:36,2014-11-21,2018-02-02
4,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-10-19 00:15:29,2015-02-12,2018-10-18


In [18]:
a[a['code'] == 'NQU1999']['name'] = 'dupa'

C:\Users\tomek\Anaconda3\envs\env_zipline\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
a.loc[a[a['code'] == 'NQU1999'].index, 'name'] = 'dupa'

In [23]:
a[a['code'] == 'NQU1999']

,code,name,description,refreshed_at,from_date,to_date
89771,NQU1999,dupa,"For contract details, please see <a href=http:...",2018-06-29 17:34:25,1999-06-21,1999-09-17


In [ ]:
a.loc['NQU1999', 'name'] = 'Nasdaq 100 Mini Futures, September 1999, NQU1999, CME'
